# 6. 금융 데이터 수집하기 (심화)

지난 장에서 수집한 주식티커를 바탕으로 이번 CHAPTER에서는 퀀트 투자의 핵심 자료인 수정주가, 재무제표, 가치지표를 크롤링하는 방법을 알아보겠습니다.

## 6.1 수정주가 크롤링

주가 데이터는 투자를 함에 있어 반드시 필요한 데이터이며, 인터넷에서 주가를 수집할 수 있는 방법은 매우 많습니다. 먼저 API를 이용한 데이터 수집에서 살펴본 것과 같이, `DataReader()` 함수를 이용해 데이터를 받을 수 있습니다. 그러나 야후 파이낸스에서 제공하는 데이터 중 미국 주가는 이상 없이 다운로드되지만, 국내 중소형주는 주가가 없는 경우가 있습니다.

또한 단순 주가를 구할 수 있는 방법은 많지만, 투자에 필요한 수정주가를 구할 수 있는 방법은 찾기 힘듭니다. 다행히 네이버 금융에서 제공하는 정보를 통해 모든 종목의 수정주가를 매우 손쉽게 구할 수 있습니다.

### 6.1.1 개별종목 주가 크롤링

```
https://finance.naver.com/item/fchart.nhn?code=005930
```
먼저 네이버 금융에서 특정종목(예: 삼성전자)의 [차트]을 선택합니다. 해당 차트는 주가 데이터를 받아 그래프를 그려주는 형태입니다. 따라서 해당 데이터가 어디에서 오는지 알기 위해 개발자 도구 화면을 이용합니다.

```{figure} image/06_crawl_practice_price2.png
---
name: 06_crawl_practice_price2
---
네이버금융 차트의 통신기록
```

화면을 연 상태에서 [일] 탭을 선택하면 나오는 항목 중 가장 상단 항목의 Request URL이 주가 데이터를 요청하는 주소입니다. 해당 URL에 접속해보겠습니다.

```
https://fchart.stock.naver.com/siseJson.nhn?symbol=005930&requestType=1&startTime=20191117&endTime=20210124&timeframe=day
```

```{figure} image/06_crawl_practice_price3.png
---
name: 06_crawl_practice_price3
---
주가 데이터 페이지
```

각 날짜별로 시가, 고가, 저가, 종가, 거래량, 외국인소진율이 있으며, 주가는 모두 수정주가 기준입니다.

URL에서 symbol= 뒤에 6자리 티커만 변경하면 해당 종목의 주가 데이터가 있는 페이지로 이동할 수 있으며, 이를 통해 우리가 원하는 모든 종목의 주가 데이터를 크롤링할 수 있습니다. 또한 startTime= 에는 시작일자를, endTime= 에는 종료일자를 입력하여 원하는 기간 만큼의 데이터를 받을 수도 있습니다.

In [1]:
import pandas as pd

KOR_ticker = pd.read_csv('data/KOR_ticker.csv', index_col=0)

In [2]:
KOR_ticker['종목코드'][0]

5930

In [3]:
import numpy as np

KOR_ticker['종목코드'] = KOR_ticker['종목코드'].astype(np.str).str.zfill(6)

먼저 저장해두었던 티커 항목의 csv 파일을 불러옵니다. 종목코드를 살펴보면 **005930**이어야 할 삼성전자의 티커가 **5930**으로 입력되어 있습니다. 이는 파일을 불러오는 과정에서 0으로 시작하는 숫자들이 지워졌기 때문입니다. 먼저 `np.str`을 통해 문자열 형태로 변경한 후, `zfill()` 함수를 사용해 6자리가 되지 않는 문자는 왼쪽에 0을 추가해 강제로 6자리로 만들어주도록 합니다.

다음은 첫 번째 종목인 삼성전자의 주가를 크롤링한 후 가공하는 방법입니다.

In [4]:
import os
import numpy as np
from datetime import date

if not os.path.exists('data/KOR_price'):
    os.makedirs('data/KOR_price')
    
i = 0
name = KOR_ticker['종목코드'][i]   

price = pd.DataFrame({'Price' : [np.nan]})
price.index = [date.today().strftime("%Y-%m-%d")]

In [5]:
price

,Price
2021-02-15,NaN


1. data 폴더 내에 KOR_price 폴더를 생성합니다.
2. i = 0을 입력합니다. 향후 for loop 구문을 통해 i 값만 변경하면 모든 종목의 주가를 다운로드할 수 있습니다.
3. name에 해당 티커를 입력합니다.
4. 빈 데이터프레임을 생성하며, 인덱스는 `today()`를 통해 현재 날짜를 입력합니다.

In [6]:
from dateutil.relativedelta import relativedelta
import requests as rq
from io import BytesIO

fr = (date.today() + relativedelta(years=-3)).strftime("%Y%m%d")
to = (date.today()).strftime("%Y%m%d")

url = 'https://fchart.stock.naver.com/siseJson.nhn?symbol='+name+'&requestType=1&startTime='+fr+'&endTime='+to+'&timeframe=day'

data = rq.get(url).content
data_price = pd.read_csv(BytesIO(data))

In [7]:
data_price.head()

,[['날짜','시가','고가','저가','종가','거래량','외국인소진율'],Unnamed: 7
0,"[""20180219""",49800.0,49800.0,47860.0,48380.0,307069.0,52.25],NaN
1,"[""20180220""",48040.0,48160.0,47220.0,47400.0,202452.0,52.23],NaN
2,"[""20180221""",47280.0,47580.0,46840.0,47280.0,257604.0,52.22],NaN
3,"[""20180222""",47260.0,47260.0,46760.0,46760.0,177399.0,52.22],NaN
4,"[""20180223""",46760.0,47800.0,46760.0,47220.0,248466.0,52.2],NaN


1. 먼저 시작일(from)과 종료일(to)에 해당하는 날짜를 입력합니다. `today()`를 통해 오늘 날짜를 불러온 후, 시작일은 `relativedelta()` 함수를 이용해 3년을 빼줍니다. (본인이 원하는 기간 만큼을 빼주면 됩니다.) 그 후 `strftime()` 함수를 통해 yyyymmdd 형식을 만들어 줍니다.
2. 종목의 url을 생성합니다. url 중 티커에 해당하는 6자리 부분에 위에서 입력한 name을 설정합니다.
3. `get()` 함수를 통해 페이지의 데이터를 불러온 후, content 부분을 추출합니다.
4. `BytesIO()` 함수를 이용해 바이너리스트림 형태로 만든 후, `read_csv()` 함수를 통해 데이터를 읽어옵니다

결과적으로 날짜 및 주가, 거래량, 외국인소진율 데이터가 추출됩니다. 우리에게 필요한 날짜와 종가에 해당하는 열만 선택하고, 클렌징 작업을 해주도록 하겠습니다.

In [8]:
import re
from datetime import datetime

price = data_price.iloc[:, [0, 4]]
price.columns = ['Date', 'Price']
price = price.dropna()
price['Date'] = price['Date'].str.extract('(\d+)')
price['Date'] = pd.to_datetime(price['Date'])
price.set_index('Date', inplace=True)

In [9]:
price.head()

,Price
Date,
2018-02-19,48380.0
2018-02-20,47400.0
2018-02-21,47280.0
2018-02-22,46760.0
2018-02-23,47220.0


1. 날짜에 해당하는 열과, 종가에 해당하는 열만 선택해 저장합니다.
2. 열 이름을 Date와 Price로 변경합니다.
3. `dropna()` 함수를 통해 NA 데이터를 삭제해줍니다.
4. `extract()` 함수 내에 정규표현식을 통해 숫자만을 추출합니다.
4. `apply()` 함수를 통해 Date열을 datetime 형태로 변경해줍니다.
5. Date 열을 인덱스로 설정합니다.

데이터를 확인해보면 우리에게 필요한 형태로 정리되었습니다.

In [ ]:
price.to_csv('data/KOR_price/'+name+'_price.csv')

마지막으로 해당 데이터를 data 폴더의 KOR_price 폴더 내에 티커_price.csv 이름으로 저장합니다.

### 6.1.2 전 종목 주가 크롤링

위의 코드에서 `for loop` 구문을 이용해 i 값만 변경해주면 모든 종목의 주가를 다운로드할 수 있습니다. 전 종목 주가를 다운로드하는 전체 코드는 다음과 같습니다.

In [ ]:
import pandas as pd
import numpy as np
from datetime import date
from datetime import datetime
import os
from dateutil.relativedelta import relativedelta
import requests as rq
from io import BytesIO
import re
import time

if not os.path.exists('data/KOR_price'):
    os.makedirs('data/KOR_price')

KOR_ticker = pd.read_csv('data/KOR_ticker.csv', index_col=0)
KOR_ticker['종목코드'] = KOR_ticker['종목코드'].astype(np.str).str.zfill(6)

for i in range(0, len(KOR_ticker)): 

    # 빈 데이터프레임 생성
    price = pd.DataFrame({'Price' : [np.nan]})
    price.index = [date.today().strftime("%Y-%m-%d")]
    
    # 티커 선택
    name = KOR_ticker['종목코드'][i]
    
    # 시작일과 종료일 
    fr = (date.today() + relativedelta(years=-3)).strftime("%Y%m%d")
    to = (date.today()).strftime("%Y%m%d")
    
    # 오류 발생 시 이를 무시하고 다음 루프로 진행
    try:
        # url 생성
        url = 'https://fchart.stock.naver.com/siseJson.nhn?symbol='+name+'&requestType=1&startTime='+fr+'&endTime='+to+'&timeframe=day'
        
        # 데이터 다운로드
        data = rq.get(url).content
        data_price = pd.read_csv(BytesIO(data)) 
        
        # 필요한 열만 선택 후 클렌징
        price = data_price.iloc[:, [0, 4]]
        price.columns = ['Date', 'Price']
        price = price.dropna()
        price['Date'] = price['Date'].str.extract('(\d+)')
        price['Date'] = pd.to_datetime(price['Date'])
        price.set_index('Date', inplace=True)
    
    except:
        # 오류 발생시 해당 종목명을 출력하고 다음 루프로 이동
        print('Error in ticker ' + name)
    
    # 다운로드 받은 파일을 생성한 폴더 내 csv 파일로 저장
    price.to_csv('data/KOR_price/'+name+'_price.csv')
    
    # 타임슬립 적용
    time.sleep(2)    

위 코드에서 추가된 점은 다음과 같습니다. 페이지 오류, 통신 오류 등 오류가 발생할 경우 for loop 구문은 멈춰버리는데 전체 데이터를 처음부터 다시 받는 일은 매우 귀찮은 작업입니다. 따라서 `try except` 구문을 이용해 오류가 발생할 때 해당 티커를 출력한 후 다음 루프로 넘어가게 합니다.

또한 오류가 발생하면 미리 만들어둔 빈 데이터프레임을 저장하게 됩니다. 마지막으로 무한 크롤링을 방지하기 위해 한 번의 루프가 끝날 때마다 2초의 타임슬립을 적용했습니다.

위 코드가 모두 돌아가는 데는 수 시간이 걸립니다. 작업이 끝난 후 data/KOR_price 폴더를 확인해보면 전 종목 주가가 csv 형태로 저장되어 있습니다.

## 6.2 재무제표 및 가치지표 크롤링

주가와 더불어 재무제표와 가치지표 역시 투자에 있어 핵심이 되는 데이터입니다. 해당 데이터 역시 여러 웹사이트에서 구할 수 있지만, 국내 데이터 제공업체인 FnGuide에서 운영하는 Company Guide 웹사이트에서 손쉽게 구할 수 있습니다.

```
http://comp.fnguide.com/
```

### 6.2.1 재무제표 다운로드

먼저 개별종목의 재무제표를 탭을 선택하면 포괄손익계산서, 재무상태표, 현금흐름표 항목이 보이게 되며, 티커에 해당하는 A005930 뒤의 주소는 불필요한 내용이므로, 이를 제거한 주소로 접속합니다. A 뒤의 6자리 티커만 변경한다면 해당 종목의 재무제표 페이지로 이동하게 됩니다.

```
http://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A005930
```
우리가 원하는 재무제표 항목들은 모두 테이블 형태로 제공되고 있으므로 pandas의 `read_html()` 함수를 이용해 추출할 수 있습니다.

In [10]:
import os
import pandas as pd

if not os.path.exists('data/KOR_fs'):
    os.makedirs('data/KOR_fs')
    
url = 'http://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A005930'
data = pd.read_html(url, displayed_only = False)

In [11]:
[item.head(3) for item in data] 

[  IFRS(연결)    2017/12    2018/12    2019/12    2020/09       전년동기  전년동기(%)
 0      매출액  2395754.0  2437714.0  2304009.0  1752555.0  1705161.0      2.8
 1     매출원가  1292907.0  1323944.0  1472395.0  1066834.0  1086850.0     -1.8
 2    매출총이익  1102847.0  1113770.0   831613.0   685721.0   618311.0     10.9,
   IFRS(연결)   2019/12   2020/03   2020/06   2020/09      전년동기  전년동기(%)
 0      매출액  598848.0  553252.0  529661.0  669642.0  620035.0      8.0
 1     매출원가  385545.0  348067.0  319062.0  399705.0  399939.0     -0.1
 2    매출총이익  213302.0  205185.0  210599.0  269937.0  220096.0     22.6,
              IFRS(연결)    2017/12    2018/12    2019/12    2020/09
 0                  자산  3017521.0  3393572.0  3525645.0  3757887.0
 1  유동자산계산에 참여한 계정 펼치기  1469825.0  1746974.0  1813853.0  2036349.0
 2                재고자산   249834.0   289847.0   267665.0   324429.0,
              IFRS(연결)    2019/12    2020/03    2020/06    2020/09
 0                  자산  3525645.0  3574575.0  3579595.0  3757887.0
 1  유동자

1. data 폴더 내에 KOR_fs 폴더를 생성합니다.
2. url을 입력한 후 `read_html()` 함수를 통해 테이블 데이터만을 가져옵니다. 페이지를 살펴보면 [+] 버튼을 눌러야만 표시가 되는 항목도 있으므로, displayed_only = FALSE를 통해 해당 항목들도 모두 가져오도록 합니다.

위의 과정을 거치면 총 6개의 테이블이 들어오게 되며, 그 내용은 {numref}`fs_table`와 같습니다.

```{table} 재무제표 테이블 내역
:name: fs_table

| 순서 | 내용 | 
| --- | --- |
| 0 | 포괄손익계산서 (연간) |
| 1 | 포괄손익계산서 (분기) |
| 2 | 재무상태표 (연간) |
| 3 | 재무상태표 (분기) |
| 4 | 현금흐름표 (연간) |
| 5 | 현금흐름표 (분기) |
```

이 중 연간 기준 재무제표에 해당하는 테이블을 선택합니다.

In [12]:
data_IS = data[0]
data_BS = data[2]
data_CF = data[4]

In [13]:
data_IS.columns

Index(['IFRS(연결)', '2017/12', '2018/12', '2019/12', '2020/09', '전년동기',
       '전년동기(%)'],
      dtype='object')

In [14]:
data_IS = data_IS.iloc[:, 0:(len(data_IS.columns)-2)]
data_fs = pd.concat([data_IS, data_BS, data_CF]).reset_index(drop=True)

data_fs.head()

,IFRS(연결),2017/12,2018/12,2019/12,2020/09
0,매출액,2395754.0,2437714.0,2304009.0,1752555.0
1,매출원가,1292907.0,1323944.0,1472395.0,1066834.0
2,매출총이익,1102847.0,1113770.0,831613.0,685721.0
3,판매비와관리비계산에 참여한 계정 펼치기,566397.0,524903.0,553928.0,416252.0
4,인건비,67972.0,64514.0,64226.0,51701.0


포괄손익계산서 테이블(data_IS)에는 전년동기, 전년동기(%) 열이 있는데 통일성을 위해 해당 열을 삭제합니다. `concat()` 함수를 이용해 세 테이블을 행으로 묶은 후 클렌징을 하겠습니다.

In [15]:
data_fs.iloc[:, 0] = data_fs.iloc[:, 0].replace({'계산에 참여한 계정 펼치기':''}, regex = True)
data_fs = data_fs.set_index(data_fs.columns[0])
data_fs = data_fs[~data_fs.index.duplicated(keep='first')]
data_fs = data_fs.filter(like = '12', axis = 1)

data_fs.head()

,2017/12,2018/12,2019/12
IFRS(연결),,,
매출액,2395754.0,2437714.0,2304009.0
매출원가,1292907.0,1323944.0,1472395.0
매출총이익,1102847.0,1113770.0,831613.0
판매비와관리비,566397.0,524903.0,553928.0
인건비,67972.0,64514.0,64226.0


1. 첫 번째 열인 계정명에는 '계산에 참여한 계정 펼치기' 라는 글자가 들어간 항목이 있습니다. 이는 페이지 내에서 펼치기 역할을 하는 [+] 항목에 해당하며 `replace()` 함수를 이용해 해당 글자를 삭제합니다.
2. 첫 번째 열의 계정명을 인덱스로 변경합니다.
3. 인덱스에는 중복되는 계정명이 다수 있는데 대부분 불필요한 항목입니다. `duplicated()` 함수를 사용해 중복되는 항목 중, 첫번째 항목만을 남겨둡니다.
4. 간혹 12월 결산법인이 아닌 종목이거나 연간 재무제표임에도 불구하고 분기 재무제표가 들어간 경우가 있습니다. 비교의 통일성을 위해 `filter()` 함수를 통해 12라는 숫자가 포함된 열만 선택합니다.

데이터를 확인해보면 연간 기준 재무제표가 정리되었습니다. 

In [ ]:
data_fs.to_csv('data/KOR_fs/005930_fs.csv')

data 폴더의 KOR_fs 폴더 내에 티커_fs.csv 이름으로 저장합니다.

### 6.2.2 가치지표 계산하기

위에서 구한 재무제표 데이터를 이용해 가치지표를 계산할 수 있습니다. 흔히 사용되는 가치지표는 **PER, PBR, PCR, PSR**이며 분자는 주가, 분모는 재무제표 데이터가 사용됩니다.

```{table} 가치지표의 종류
:name: value_table

| 지표 | 분모 | 
| --- | --- |
| PER | Earnings (순이익) |
| PBR | Book Value (순자산) |
| PCR | Cashflow (영업활동현금흐름) |
| PSR | Sales (매출액) |
```

위에서 구한 재무제표 항목에서 분모 부분에 해당하는 데이터만 선택해보겠습니다.

In [16]:
if not os.path.exists('data/KOR_value'):
    os.makedirs('data/KOR_value')
    
value_type = ['지배주주순이익', '자본', '영업활동으로인한현금흐름', '매출액']
value_index = data_fs.loc[value_type].iloc[:, -1:]
print(value_index)

                2019/12
IFRS(연결)               
지배주주순이익        215051.0
자본            2628804.0
영업활동으로인한현금흐름   453829.0
매출액           2304009.0


1. data 폴더 내에 KOR_value 폴더를 생성합니다.
2. 분모에 해당하는 항목을 value_type에 저장한 후 재무제표 항목에서 해당 데이터를 찾으며, 맨 오른쪽 즉 최근년도 재무제표 데이터를 선택합니다.

다음으로 가치지표 계산을 위해 종가와 발행주식수를 수집해야 합니다. 예를 들어 PER를 계산하는 방법은 다음과 같습니다.

$$PER = \frac{Price}{EPS} = \frac{주가}{주당순이익}$$

주당순이익은 순이익을 발행주식수로 나눈 값이며, 해당 값의 계산하려면 주가와 발행주식수를 추가로 구해야 합니다. 이 또한 Company Guide 사이트에서 구할 수 있으며, 불필요한 부분을 제거한 URL은 다음과 같습니다.

```
http://comp.fnguide.com/SVO2/ASP/SVD_main.asp?pGB=1&gicode=A005930
```

위의 주소 역시 A 뒤의 6자리 티커만 변경하면 해당 종목의 스냅샷 페이지로 이동하게 됩니다.

```{figure} image/05_crawl_practice_comp_price.png
---
name: 05_crawl_practice_comp_price
---
Company Guide 스냅샷 화면
```



시세현황표 내에 종가와 발행주식수가 있으므로, 이를 크롤링하겠습니다.

In [17]:
url = 'http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A005930'
data = pd.read_html(url)[0]

data

,0,1,2,3
0,종가/ 전일대비,"81,600/ -1,100",거래량,23025766
1,52주.최고가/ 최저가,"91,000/ 42,500",거래대금(억원),18886
2,수익률(1M/ 3M/ 6M/ 1Y),-8.11/ +35.55/ +41.18/ +36.68,외국인 보유비중,54.82
3,"시가총액(상장예정포함,억원)",5482747,베타(1년),1.00569
4,"시가총액(보통주,억원)",4871343,액면가,100
5,NaN,NaN,NaN,NaN
6,발행주식수(보통주/ 우선주),"5,969,782,550/ 822,886,700",유동주식수/비율(보통주),"4,460,337,489 / 74.72"


In [18]:
price = data.iloc[0, 1].split()[0]
price = price.replace(",", "")
price = int(re.findall("[0-9]+", price)[0])

price

81600

1. [종가/전일대비]의 숫자에 해당하는 부분을 선택합니다. `split()` 함수를 통해 데이터를 나눈 후, 종가에 해당하는 첫번째 데이터를 선택합니다.
2. 쉼표(,)를 없애줍니다.
3. `findall()` 함수 내부에 정규표현식을 통해 숫자 부분만을 선택하고, 이를 정수형으로 변경해줍니다.

이를 통해 종가에 해당하는 부분이 선택되었습니다. 보통주의 발행주식수 역시 이와 유사한 방법으로 구할 수 있습니다.

In [19]:
share = data.iloc[6, 1].split()[0]
share = share.replace(",", "")
share = int(re.findall("[0-9]+", share)[0])

share

5969782550

이제 재무 데이터, 현재 주가, 발행주식수를 이용해 가치지표를 계산해보겠습니다.

In [20]:
import numpy as np

data_value = price / (value_index * 100000000 / share)
data_value.index = ['PER', 'PBR', 'PCR', 'PSR']
data_value[data_value.iloc[:, 0] < 0] = np.nan

data_value

,2019/12
PER,22.652034
PBR,1.853064
PCR,10.733872
PSR,2.114290


분자에는 현재 주가를 입력하며, 분모에는 재무 데이터를 보통주 발행주식수로 나눈 값을 입력합니다. 단, 주가는 원 단위, 재무 데이터는 억 원 단위이므로, 둘 사이에 단위를 동일하게 맞춰주기 위해 분모에 억을 곱합니다. 또한 가치지표가 음수인 경우는 NA로 변경해줍니다.

결과를 확인해보면 4가지 가치지표가 잘 계산되었습니다.[^value]

[^value]: 분모에 사용되는 재무데이터의 구체적인 항목과 발행주식수를 계산하는 방법의 차이로 인해 여러 업체에서 제공하는 가치지표와 다소 차이가 발생할 수 있습니다.

In [ ]:
data_value.to_csv('data/KOR_value/005930_value.csv')

data 폴더의 KOR_value 폴더 내에 티커_value.csv 이름으로 저장합니다.

### 6.2.3 전 종목 재무제표 및 가치지표 다운로드

위 코드에서 for loop 구문을 이용해 URL 중 6자리 티커에 해당하는 값만 변경해주면 모든 종목의 재무제표를 다운로드하고 이를 바탕으로 가치지표를 계산할 수 있습니다. 해당 코드는 다음과 같습니다.

In [ ]:
import pandas as pd
import numpy as np
import os
import time
import re

if not os.path.exists('data/KOR_fs'):
    os.makedirs('data/KOR_fs')
    
if not os.path.exists('data/KOR_value'):
    os.makedirs('data/KOR_value')

KOR_ticker = pd.read_csv('data/KOR_ticker.csv', index_col=0)
KOR_ticker['종목코드'] = KOR_ticker['종목코드'].astype(np.str).str.zfill(6)

for i in range(0, len(KOR_ticker)):
    
    # 빈 데이터프레임 생성
    data_fs = pd.DataFrame({'' : [np.nan]})
    data_value = pd.DataFrame({'' : [np.nan]})
    
    # 티커 선택
    name = KOR_ticker['종목코드'][i]
    
    # 오류 발생 시 이를 무시하고 다음 루프로 진행
    try:
        # url 생성
        url = 'http://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A'+name
        data = pd.read_html(url, displayed_only = False)
        
        # 데이터 다운로드 후 테이블 추출
        data = pd.read_html(url)
                
        data_IS = data[0]
        data_BS = data[2]
        data_CF = data[4]
        data_IS = data_IS.iloc[:, 0:(len(data_IS.columns)-2)]
        
        # 클렌징
        data_fs = pd.concat([data_IS, data_BS, data_CF]).reset_index(drop=True)
        data_fs.iloc[:, 0] = data_fs.iloc[:, 0].replace({'계산에 참여한 계정 펼치기':''}, regex = True)
        data_fs = data_fs.set_index(data_fs.columns[0])
        data_fs = data_fs[~data_fs.index.duplicated(keep='first')]
        data_fs = data_fs.filter(like = '12', axis = 1)
        
        # 가치 지표 선택
        value_type = ['지배주주순이익', '자본', '영업활동으로인한현금흐름', '매출액']            
        value_index = data_fs.loc[value_type].iloc[:, -1:]  
        
        # 현재 주가
        url_snap = 'http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A'+name
        data_snap = pd.read_html(url_snap)[0]
        
        price = data_snap.iloc[0, 1].split()[0]
        price = price.replace(",", "")
        price = int(re.findall("[0-9]+", price)[0])
        
        # 상장 주식수
        share = data_snap.iloc[6, 1].split()[0]
        share = share.replace(",", "")
        share = int(re.findall("[0-9]+", share)[0])
        
        data_value = price / (value_index * 100000000 / share)
        data_value.index = ['PER', 'PBR', 'PCR', 'PSR']
        data_value[data_value.iloc[:, 0] < 0] = np.nan     
        
    except:
        # 오류 발생시 해당 종목명을 출력하고 다음 루프로 이동
        # count_ticker에 오류발생 종목 저장, 마감 후 원인 파악
        print('Error in ticker ' + name)        
    
    # 다운로드 받은 파일을 생성한 폴더 내 csv 파일로 저장    
    data_fs.to_csv('data/KOR_fs/'+name+'_fs.csv')
    data_value.to_csv('data/KOR_value/'+name+'_value.csv')
    
    # 타임슬립 적용
    time.sleep(2)